In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import accuracy_score, balanced_accuracy_score, recall_score, precision_score, confusion_matrix
import numpy as np
import pandas as pd
from utils import specificity_score, negative_prediction_value_score, gmean_score, informedness_score
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from base_pulsa_ai import BasePulsarAI

# Data Loading

In [16]:
base_pulsar_ai = BasePulsarAI()
x_train, x_test, y_train, y_test = base_pulsar_ai.get_torch_htru_2()

# Model definition

In [17]:
class ClassicCNN(nn.Module):
    def __init__(self, num_features, output_dim):
        super(ClassicCNN, self).__init__()
        self.conv1d = nn.Conv1d(in_channels=num_features, out_channels=128, kernel_size=1)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = x.view(-1, 8, 1)
        x = self.conv1d(x)
        x = self.relu(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = torch.sigmoid(x)
        x = x.squeeze(1)
        return x


input_dim  = 8 # number of features
output_dim = 2 # binary classification, to be change for sigmoid ?

model = ClassicCNN(input_dim, output_dim)
output_train = model(x_train)

print(output_train)

tensor([0.9923, 0.9911, 0.9747,  ..., 0.9950, 0.9946, 0.9943],
       grad_fn=<SqueezeBackward1>)


# Training CNN

In [21]:


ss = ShuffleSplit(n_splits=base_pulsar_ai.num_runs, train_size=60, test_size=120)

def train_network(model,optimizer,criterion,x_train,y_train,x_test,y_test,num_epochs,train_losses,test_losses):
    for epoch in range(num_epochs):
        #clear out the gradients from the last step loss.backward()
        optimizer.zero_grad()
        
        #forward feed
        output_train = model(x_train.float())

        #calculate the loss
        loss_train = criterion(output_train, y_train.float())
        
        #backward propagation: calculate gradients
        loss_train.backward()

        #update the weights
        optimizer.step()
        
        output_test = model(x_test)
        loss_test = criterion(output_test,y_test.float())

        train_losses[epoch] = loss_train.item()
        test_losses[epoch] = loss_test.item()

for train_index, test_index in ss.split(x_train):
    num_epochs = base_pulsar_ai.max_num_epochs
    train_losses = np.zeros(num_epochs)
    test_losses  = np.zeros(num_epochs)
    model = ClassicCNN(input_dim, output_dim)

    learning_rate = 0.01
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

    train_network(model,optimizer,criterion, x_train,y_train, x_test,y_test,num_epochs,train_losses,test_losses)

    # Predict the test set
    output_test = model(x_test)
    _, predicted_test = torch.max(output_test, 1)

    # Calculate the scores
    base_pulsar_ai.append_score(y_test, predicted_test)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

# Prediction

In [ ]:
# Print the scores
for metric, values in base_pulsar_ai.scores.items():
    mean_value = np.mean(values)
    std_value = np.std(values)
    print(f"{metric.capitalize()}: {mean_value:.3f} ± {std_value:.3f}")

Accuracy: 0.976 ± 0.000
Balanced_accuracy: 0.904 ± 0.001
Recall: 0.815 ± 0.002
Specificity: 0.993 ± 0.000
Precision: 0.924 ± 0.003
Npv: 0.981 ± 0.000
Gmean: 0.900 ± 0.001
Informedness: 0.808 ± 0.003
